In [1]:
from utils import *

In [2]:
blog_author_regex = "<wp:author_login><!\[CDATA\[(.*?)\]\]>.*<wp:author_display_name><!\[CDATA\[(.*?)]]><\/wp:author_display_name>.*<\/wp:author>"
mag_author_regex = "<!\[CDATA\[_issuem_author_name\]\]><\/wp:meta_key>[\s\S]*?<wp:meta_value><!\[CDATA\[(.*?)\]\]"

media_regex = "<item>[\s\S]*?<guid .*?>(.*?)<\/guid>[\s\S]*?<wp:post_parent>(.*?)<\/wp:post_parent>[\s\S]*?<\/item>"
blog_regex = "<item>[\s\S]*?<title>(.*?)<\/title>[\s\S]*?<link>(.*?)<\/link>[\s\S]*?<dc:creator><!\[CDATA\[(.*?)\]\]><\/dc:creator>[\s\S]*?p=(.*?)<\/guid>[\s\S]*?<content:encoded><!\[CDATA\[([\s\S]*?)\]\][\s\S]*?<wp:post_date><!\[CDATA\[(.*?)\]\]><\/wp:post_date>[\s\S]*?<\/item>"
mag_regex = '<item>[\s\S]*?<title>(.*?)<\/title>[\s\S]*?<link>(.*?)<\/link>[\s\S]*?<guid[\s\S]*?p=(.*?)<\/guid>[\s\S]*?<content:encoded><!\[CDATA\[([\s\S]*?)\]\]><\/content:encoded>[\s\S]*?<wp:post_date><!\[CDATA\[(.*?)\]\]><\/wp:post_date>[\s\S]*?<category domain="issuem_issue"[\s\S]*?<!\[CDATA\[(.*?)\]\][\s\S]*?<!\[CDATA\[_issuem_author_name\]\]><\/wp:meta_key>[\s\S]*?<wp:meta_value><!\[CDATA\[(.*?)\]\][\s\S]*?<\/item>'

authors = {}

with open('posts/authors.xml', 'r') as xml:
    contents = xml.read()
    for i, auth_display_name in re.findall(blog_author_regex, contents):
        authors[i] = auth_display_name

with open('posts/magazine-articles.xml', 'r') as xml:
    contents = xml.read()
    for auth_display_name in re.findall(mag_author_regex, contents):
        authors[auth_display_name] = auth_display_name

media = {}
        
with open('posts/media.xml', 'r') as xml:
    contents = xml.read()
    for url, parent_id in re.findall(media_regex, contents):
        media[parent_id] = url        
        
authors_df = pd.DataFrame(authors.values()).replace('', np.nan).dropna().rename(columns={0: "name"})
article_title_categories = pd.read_csv('posts/article-categories2.csv')
categories = pd.read_csv('posts/categories.csv', header=None)

authors_df
# authors_df.to_json('authors.js', orient='records', indent=1)
# categories.to_json('categories.js', orient='records', indent=1)

,name
0,Liz Boatman
1,Jessica Harvey
2,Anna Goldstein
3,Rachel Bernstein
4,Brian Lambson
...,...
317,Andrea Herman
318,Mattina Alonge
319,Orr Paradise
320,Emma Regan


In [ ]:
blog_posts = []

with open('posts/export.xml', 'r') as xml:
    contents = xml.read()
    draw_progress_bar(0)
    matches = re.findall(blog_regex, contents)
    for i, post in enumerate(matches):
        blog_posts.append({})
        ID = post[3]

        blog_posts[-1]["id"] = ID
        blog_posts[-1]["bsr_permalink"] = post[1]
        blog_posts[-1]["title"] = markdownify(post[0])
        blog_posts[-1]["author"] = authors[post[2]]
        
        
        if post[5]:
            blog_posts[-1]["created_at"] = parser.parse(post[5]).strftime('%Y-%m-%dT%H:%M:%S.000Z')
            
        blog_posts[-1]['issue'] = np.nan
        
#         if ID in media:
#             blog_posts[-1]['featured_image'] = media[ID]
#             flag = 'featured_image given'
            
#         else:
#             blog_posts[-1]['featured_image'], flag = retrieve_img(post[1])

        blog_posts[-1]['featured_image'], flag = retrieve_img(post[1])
            
        newlines = re.sub('\n', '<br><br>', post[4])
        hlines = re.sub('<hr />', '-------', newlines)
        out = re.sub('<!-- .*? -->', '', hlines)
        final = re.sub(r'\[caption .*?\](.*?\/>) (.*?)\[\/caption\]', r'\1<br><em>\2</em>', out)

        md = markdownify(final, heading_style=ATX).lstrip()
        final = re.sub('\n\s*\n', '<br><br>', md)
#         final = re.sub(r'\[caption .*?src="(.*?)"[\s\S]*?\[\/caption\]',r'<img src="\1"/>',final)
#         print(final)
        
        blog_posts[-1]["markdown"] = markdownify(final, heading_style=ATX).lstrip()

        draw_progress_bar(i/len(matches), flag)

blog_posts = pd.DataFrame(blog_posts)
blog_posts = blog_posts[~blog_posts['bsr_permalink'].str.contains("trashed")]

[                                        ]   2%       'NoneType' object has no attribute 'find'

In [ ]:
articles = []

with open('posts/magazine-articles.xml', 'r') as xml:
    contents = xml.read() 
    draw_progress_bar(0)
    matches = re.findall(mag_regex, contents)
    for i, article in enumerate(matches):
        articles.append({})
        ID = article[2]

        articles[-1]["id"] = ID
        
        articles[-1]["bsr_permalink"] = article[1]
        articles[-1]["title"] = markdownify(article[0])
        articles[-1]["author"] = authors[article[6]]
        
        if article[5]:
            articles[-1]["created_at"] = parser.parse(article[4]).strftime('%Y-%m-%dT%H:%M:%S.000Z')

        articles[-1]['issue'] = article[5]
            
#         if ID in media:
#             articles[-1]['featured_image'] = media[ID]
#             flag = 'featured_image given'
#         else:
#             articles[-1]['featured_image'], flag = retrieve_img(article[1])
        articles[-1]['featured_image'], flag = retrieve_img(article[1])
    
        newlines = re.sub('\n', '<br><br>', article[3])
        hlines = re.sub('<hr />', '-------', newlines)
        out = re.sub('<!-- .*? -->', '', hlines)
        final = re.sub(r'\[caption .*?\].*?(<img.*?\/>).*?<\/a> .*?(.*?)\[\/caption\]', r'\1<br><em>\2</em>', out)
        final = re.sub(r'\[caption .*?\](.*?\/>) (.*?)\[\/caption\]', r'\1<br><em>\2</em>', final)
        

        md = markdownify(final, heading_style=ATX).lstrip()
        final = re.sub('\n\s*\n', '<br><br>', md)
        
        articles[-1]["markdown"] = markdownify(final, heading_style=ATX).lstrip()

        draw_progress_bar(i/len(matches), flag)

articles = pd.DataFrame(articles)
articles = articles[~articles['bsr_permalink'].str.contains("trashed")]

In [ ]:
merged = pd.concat([blog_posts,articles],axis=0)

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


def fuzzy_merge(df_1, df_2, key1, key2, threshold=95, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    c = df_1['matches'].apply(lambda x: ', '.join([str(i[1]) for i in x]))
    df_1['matches'] = m2
    df_1['confidence'] = c

    return df_1

In [ ]:
articles_w_categories_fuzzy = fuzzy_merge(article_title_categories, merged, 'title', 'title', threshold=90)

In [ ]:
articles_w_categories_fuzzy = articles_w_categories_fuzzy[articles_w_categories_fuzzy.confidence.astype(int)>95]
fuzzy = articles_w_categories_fuzzy.drop(['title','confidence'], axis=1).rename(columns={"matches": "title"})

In [ ]:
final_article_df = merged.merge(fuzzy, on='title', how='left').sort_values(by="created_at", key=pd.to_datetime)

In [ ]:
final_article_df

In [41]:
print(final_article_df.loc[22, 'bsr_permalink'])

https://berkeleysciencereview.com/2010/07/welcome-to-the-bsr-blog/


In [31]:
url = 'https://berkeleysciencereview.com/wp-content/uploads/2020/12/BSRBlog_InfoInterviews_Fig3.png'
base_name = url.split('/')[-1]
img_data = requests.get(url).content

with open('images/'+base_name, 'wb') as handler:
    handler.write(img_data)

In [50]:
# if flag == 'Success':
#     img_data = requests.get(url).content
#     base_name = url.split('/')[-1]
#     blog_posts[-1]['featured_image'] = base_name

#     with open('images/'+base_name, 'wb') as handler:
#         handler.write(img_data)

In [14]:
for i, row in final_article_df.iterrows():
    if row.featured_image is not np.nan:
        url = row.featured_image
        base_name = url.split('/')[-1]
        try:
            img_data = requests.get(url).content
            with open('images/'+base_name, 'wb') as handler:
                handler.write(img_data)
        except Exception as e:
            print(e)

In [15]:
for i, row in final_article_df.iterrows():
    if row.primary_category is np.nan and parser.parse(row.created_at).year < 2018:
        row.primary_category = 'Archive'

final_article_df.to_json('export-2021-01-30.js', orient='records', indent=1)

In [16]:
pd.DataFrame(pd.unique(final_article_df['author'].dropna())).rename(columns={0: "name"}).to_json('authors.js', orient='records', indent=1)
pd.DataFrame(pd.unique(final_article_df['issue'].dropna())).rename(columns={0: "name"}).to_json('issues.js', orient='records', indent=1)
pd.DataFrame(pd.unique(final_article_df['primary_category'].dropna())).rename(columns={0: "name"}).to_json('categories.js', orient='records', indent=1)


In [17]:
final_article_df.primary_category.value_counts()

Archive               867
Life Science           53
Labscopes              32
Noteworthy News        30
Tech/AI                27
Climate Change         22
The Scientist Life     20
The Universe           16
People                  6
The universe            5
Life science            2
Climate change          1
Name: primary_category, dtype: int64

In [18]:
final_article_df

,id,bsr_permalink,title,author,created_at,issue,featured_image,markdown,primary_category,secondary_category
22,6,https://berkeleysciencereview.com/2010/07/welc...,Welcome to the BSR blog!,Anna Goldstein,2010-07-11T23:11:55.000Z,NaN,NaN,"My name is Anna, and I’m your friendly neighbo...",Archive,NaN
20,14,https://berkeleysciencereview.com/2010/07/cal-...,Cal Academy NightLife goes Extreme…Mammals,Liz Boatman,2010-07-16T12:36:09.000Z,NaN,NaN,"[caption id=""attachment\\_16"" align=""alignrigh...",Archive,NaN
21,17,https://berkeleysciencereview.com/2010/07/desi...,Designer genes,Jessica Harvey,2010-07-22T21:04:45.000Z,NaN,NaN,Synthetic biology is a promising field of rese...,Archive,NaN
40,19,https://berkeleysciencereview.com/2010/07/succ...,Success from the Neanderthal Genome Project,Anna Goldstein,2010-07-30T18:56:43.000Z,NaN,NaN,"[caption id=""attachment\\_20"" align=""alignleft...",Archive,NaN
41,22,https://berkeleysciencereview.com/2010/08/plan...,Planting the SEED for Science Education,Liz Boatman,2010-08-07T17:59:05.000Z,NaN,NaN,"[caption id=""attachment\\_29"" align=""alignrigh...",Archive,NaN
...,...,...,...,...,...,...,...,...,...,...
1116,17837,https://berkeleysciencereview.com/article/powe...,Powering down,Leela Velautham,2020-11-30T02:22:53.000Z,Fall 2020,https://berkeleysciencereview.com/wp-content/u...,When plans were first proposed to convert the ...,Climate Change,NaN
1126,17870,https://berkeleysciencereview.com/article/staf...,Staff Listing-Fall 2020,,2020-11-30T06:13:49.000Z,Fall 2020,NaN,## Editor in Chief \n \nHayley McCausland \...,NaN,NaN
755,18035,https://berkeleysciencereview.com/2020/12/issu...,Issue 39 of the Berkeley Science Review is out...,Maiko Kitaoka,2020-12-01T01:08:04.000Z,NaN,https://berkeleysciencereview.com/wp-content/u...,The Fall 2020 issue of the Berkeley Science Re...,Noteworthy News,NaN
756,18117,https://berkeleysciencereview.com/2020/12/cali...,"California Public Lands: Past, Present and Future",Laura Treers,2020-12-06T06:16:21.000Z,NaN,https://berkeleysciencereview.com/wp-content/u...,“Wilderness is not a luxury but a necessity of...,Climate Change,NaN
